### Konstantina Andronikou

## This Notebook implements a comparison between Active and Passive

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import csv

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Active 

In [4]:
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Someone drove Mary to school")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'drove',
   'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARG2: to school]',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2']}],
 'words': ['Someone', 'drove', 'Mary', 'to', 'school']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone drove Mary to school"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'drove',
     'description': '[ARG0: Someone] [V: drove] [ARG1: Mary] [ARG2: to school]',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'B-ARG2', 'I-ARG2']}],
   'words': ['Someone', 'drove', 'Mary', 'to', 'school']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} drove John to school.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Louise'}, {'first_name': 'Roger'}, {'first_name': 'Kenneth'}, {'first_name': 'Sarah'}, {'first_name': 'Kathy'}, {'first_name': 'Patricia'}, {'first_name': 'Ed'}, {'first_name': 'Christine'}, {'first_name': 'Melissa'}, {'first_name': 'Eleanor'}, {'first_name': 'Betty'}, {'first_name': 'Jonathan'}, {'first_name': 'Benjamin'}, {'first_name': 'Kathy'}, {'first_name': 'Kenneth'}, {'first_name': 'Matthew'}, {'first_name': 'Leslie'}, {'first_name': 'Caroline'}, {'first_name': 'Carolyn'}, {'first_name': 'Anne'}, {'first_name': 'Lynn'}, {'first_name': 'Joseph'}, {'first_name': 'Christopher'}, {'first_name': 'Sally'}, {'first_name': 'Nick'}, {'first_name': 'Ken'}, {'first_name': 'Diana'}, {'first_name': 'Catherine'}, {'first_name': 'Pamela'}, {'first_name': 'Judith'}, {'first_name': 'Edwin'}, {'first_name': 'Dorothy'}, {'first_name': 'Frances'}, {'first_name': 'Dorothy'}, {'first_name': 'Louis'}, {'first_name': 'Michael'}, {'first_name': 'Lauren'}, {'first_name': 'Gary'}, {

In [12]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Active_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Louise', 'drove', 'John', 'to', 'school', '.'] [ True]
['Roger', 'drove', 'John', 'to', 'school', '.'] [ True]
['Kenneth', 'drove', 'John', 'to', 'school', '.'] [ True]
['Sarah', 'drove', 'John', 'to', 'school', '.'] [ True]
['Kathy', 'drove', 'John', 'to', 'school', '.'] [ True]
['Patricia', 'drove', 'John', 'to', 'school', '.'] [ True]
['Ed', 'drove', 'John', 'to', 'school', '.'] [ True]
['Christine', 'drove', 'John', 'to', 'school', '.'] [ True]
['Melissa', 'drove', 'John', 'to', 'school', '.'] [ True]
['Eleanor', 'drove', 'John', 'to', 'school', '.'] [ True]
['Betty', 'drove', 'John', 'to', 'school', '.'] [ True]
['Jonathan', 'drove', 'John', 'to', 'school', '.'] [ True]
['Benjamin', 'drove', 'John', 'to', 'school', '.'] [ True]
['Kathy', 'drove', 'John', 'to', 'school', '.'] [ True]
['Kenneth', 'drove', 'John', 'to', 'school', '.'] [ True]
['Matthew', 'drove', 'John', 'to', 'school', '.'] [ True]
['Leslie',

In [13]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Active_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## PASSIVE

In [14]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description'] #second verb

In [15]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [16]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']#second verb
        
    return predicate_structure

In [17]:
def found_arg1_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [18]:
editor = Editor()

# create examples
t = editor.template('{first_name} got driven to school by John.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Hugh'}, {'first_name': 'Harriet'}, {'first_name': 'Tim'}, {'first_name': 'Kenneth'}, {'first_name': 'Jack'}, {'first_name': 'Bob'}, {'first_name': 'Joseph'}, {'first_name': 'Jonathan'}, {'first_name': 'Walter'}, {'first_name': 'Cynthia'}, {'first_name': 'Cynthia'}, {'first_name': 'Jonathan'}, {'first_name': 'Bill'}, {'first_name': 'Laura'}, {'first_name': 'Johnny'}, {'first_name': 'James'}, {'first_name': 'Al'}, {'first_name': 'Adam'}, {'first_name': 'Kenneth'}, {'first_name': 'Betty'}, {'first_name': 'Frank'}, {'first_name': 'Leslie'}, {'first_name': 'Dick'}, {'first_name': 'Frances'}, {'first_name': 'Judith'}, {'first_name': 'Alan'}, {'first_name': 'Arthur'}, {'first_name': 'Alice'}, {'first_name': 'Carl'}, {'first_name': 'Patricia'}, {'first_name': 'Caroline'}, {'first_name': 'Elizabeth'}, {'first_name': 'Janet'}, {'first_name': 'Bob'}, {'first_name': 'Amanda'}, {'first_name': 'Nick'}, {'first_name': 'Nick'}, {'first_name': 'Katherine'}, {'first_name': 'Edwin'}

In [19]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Passive_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Hugh', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Harriet', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Tim', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Kenneth', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Jack', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Bob', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Joseph', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Jonathan', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Walter', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Cynthia', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Cynthia', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Jonathan', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Bill', 'got', 'driven', 'to', 'school', 'by', 'John', '.'] [ True]
['Laura'

In [20]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Passive_BERT.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 